In [1]:
import pyodbc
import json
import xlrd
import xml.etree.ElementTree as ET
import xmltodict
import pprint
import xml.dom.minidom
import mysql.connector

In [2]:
with open('prequalresult.json') as json_file:
    data = json.load(json_file)

In [3]:
loan_app_id = data[0]['fields']['loanapp_id']
xml_data = (data[0]['fields']['xml_data'])
result = data[0]['fields']['result']
error_reason = data[0]['fields']['error_reason']
detail_results = data[0]['fields']['detail_results']
fico_v2 = data[0]['fields']['fico_v2']
report_type = data[0]['fields']['report_type']
username = data[0]['fields']['username']
created_at = data[0]['fields']['created_at']
personal_total_monthly_payments = data[0]['fields']['personal_total_monthly_payments']
personal_credit_card_total_limits = data[0]['fields']['personal_credit_card_total_limits']
personal_credit_card_total_balances = data[0]['fields']['personal_credit_card_total_balances']
business_credit_card_total_limits = data[0]['fields']['business_credit_card_total_limits']
business_credit_card_total_balances = data[0]['fields']['business_credit_card_total_balances']
business_total_mortgage_monthly_payments = data[0]['fields']['business_total_mortgage_monthly_payments']
business_total_monthly_debt_payments = data[0]['fields']['business_total_monthly_debt_payments']
bkdate = data[0]['fields']['bkdate']
fico_v3 = data[0]['fields']['fico_v3']
total_mortgage_monthly_payments = data[0]['fields']['total_mortgage_monthly_payments']
version = data[0]['fields']['version']
role = data[0]['fields']['role']

In [4]:
print(xml_data)
NoneType = type(None)
isinstance(personal_total_monthly_payments, NoneType)

<?xml version="1.0" standalone="no"?><NetConnectResponse xmlns="http://www.experian.com/NetConnectResponse"><CompletionCode>0000</CompletionCode><ReferenceId>testing 123</ReferenceId><TransactionId>41588980</TransactionId><Products xmlns="http://www.experian.com/ARFResponse"><CreditProfile><Header><ReportDate>10032017</ReportDate><ReportTime>140607</ReportTime><Preamble>TWA1</Preamble><ARFVersion>07</ARFVersion></Header><RiskModel><ModelIndicator code="F "/><Score>0808</Score><ScoreFactorCodeOne>10</ScoreFactorCodeOne><ScoreFactorCodeTwo>09</ScoreFactorCodeTwo><ScoreFactorCodeThree>05</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel><RiskModel><ModelIndicator code="AA"/><Score>0807</Score><ScoreFactorCodeOne>30</ScoreFactorCodeOne><ScoreFactorCodeTwo>05</ScoreFactorCodeTwo><ScoreFactorCodeThree>11</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel><ConsumerIdentity><Name><Surname>SCALI

True

In [5]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="password",
  database = "CFdb"
)
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE feilds (loan_app_id INTEGER, result BOOLEAN, error_reason VARCHAR(225), detail_results VARCHAR(800), fico_v2 VARCHAR(50), report_type VARCHAR(100), username VARCHAR(100), created_at VARCHAR(100), personal_total_monthly_payments FLOAT,personal_credit_card_total_limits FLOAT, personal_credit_card_total_balances FLOAT, business_credit_card_total_limits FLOAT, business_credit_card_total_balances FLOAT, business_total_mortgage_monthly_payments FLOAT, business_total_monthly_debt_payments FLOAT, bkdate VARCHAR(30), fico_v3 VARCHAR(20), total_mortgage_monthly_payments FLOAT, version VARCHAR(20), role VARCHAR(100))")

ProgrammingError: 1050 (42S01): Table 'feilds' already exists

In [83]:
sql = """INSERT INTO feilds (loan_app_id,result, error_reason,detail_results, fico_v2, report_type, username, created_at, personal_total_monthly_payments,personal_credit_card_total_limits,personal_credit_card_total_balances,business_credit_card_total_limits, business_credit_card_total_balances, business_total_mortgage_monthly_payments, business_total_monthly_debt_payments, bkdate, fico_v3, total_mortgage_monthly_payments, version,role) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
var = (loan_app_id,result, error_reason,detail_results, fico_v2, report_type, username, created_at, personal_total_monthly_payments,personal_credit_card_total_limits,personal_credit_card_total_balances,business_credit_card_total_limits, business_credit_card_total_balances, business_total_mortgage_monthly_payments, business_total_monthly_debt_payments, bkdate, fico_v3, total_mortgage_monthly_payments, version,role)
mycursor.execute(sql,var)
mydb.commit()

In [57]:
#doc = xml.dom.minidom.parseString(xml_data)
#doc.getElementsByTagName("Score")
#l =[p.firstChild.wholeText for p in doc.getElementsByTagName("Score") if p.firstChild.nodeType == p.TEXT_NODE]
#print(l)

[<DOM Element: Score at 0x113ab9190>, <DOM Element: Score at 0x113ab94b0>]

In [56]:
mycursor.execute("CREATE TABLE NetConnectResponse (CompletionCode INTEGER, ReferenceId VARCHAR(255), TransactionId VARCHAR(10))")

In [62]:
doc = xml.dom.minidom.parseString(xml_data)
CompletionCode = doc.getElementsByTagName("CompletionCode")[0].firstChild.wholeText
ReferenceId = doc.getElementsByTagName("ReferenceId")[0].firstChild.wholeText
TransactionId = doc.getElementsByTagName("TransactionId")[0].firstChild.wholeText
sql = """INSERT INTO NetConnectResponse (CompletionCode,ReferenceId,TransactionId) VALUES(%s,%s,%s)"""
var = (CompletionCode,ReferenceId,TransactionId)
mycursor.execute(sql,var)
mydb.commit()

In [72]:
mycursor.execute("CREATE TABLE Header (TransactionId VARCHAR(10) PRIMARY KEY, ReportDate VARCHAR(20),ReportTime VARCHAR(20),Preamble VARCHAR(20),ARFVersion VARCHAR(10))")

In [81]:
ReportDate = doc.getElementsByTagName("Header")[0].childNodes[0].firstChild.wholeText
ReportTime =doc.getElementsByTagName("Header")[0].childNodes[1].firstChild.wholeText
Preamble = doc.getElementsByTagName("Header")[0].childNodes[2].firstChild.wholeText
ARFVersion = doc.getElementsByTagName("Header")[0].childNodes[3].firstChild.wholeText
sql= """INSERT INTO Header (TransactionId,ReportDate,ReportTime,Preamble,ARFVersion) VALUES (%s,%s,%s,%s,%s)"""
var = (TransactionId,ReportDate,ReportTime,Preamble,ARFVersion)
mycursor.execute(sql,var)
mydb.commit()

In [85]:
mycursor.execute("CREATE TABLE RiskModel (TransactionId VARCHAR(10), M_code VARCHAR(10),Score VARCHAR(10),ScoreFactorCodeOne VARCHAR(10),ScoreFactorCodeTwo VARCHAR(10),ScoreFactorCodeThree VARCHAR(10),ScoreFactorCodeFour VARCHAR(10),E_code VARCHAR(10))")

In [97]:
M_code = doc.getElementsByTagName("RiskModel")[0].childNodes[0].getAttribute("_code")
Score= doc.getElementsByTagName("RiskModel")[0].childNodes[1].firstChild.wholeText
ScoreFactorCodeOne = doc.getElementsByTagName("RiskModel")[0].childNodes[2].firstChild.wholeText
ScoreFactorCodeTwo = doc.getElementsByTagName("RiskModel")[0].childNodes[3].firstChild.wholeText
ScoreFactorCodeThree = doc.getElementsByTagName("RiskModel")[0].childNodes[4].firstChild.wholeText
ScoreFactorCodeFour = doc.getElementsByTagName("RiskModel")[0].childNodes[5].firstChild.wholeText
E_code = doc.getElementsByTagName("RiskModel")[0].childNodes[6].getAttribute("_code")
sql = """INSERT INTO RiskModel (TransactionId,M_code,Score,ScoreFactorCodeOne,ScoreFactorCodeTwo,ScoreFactorCodeThree,ScoreFactorCodeFour,E_code) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
var = (TransactionId,M_code,Score,ScoreFactorCodeOne,ScoreFactorCodeTwo,ScoreFactorCodeThree,ScoreFactorCodeFour,E_code)
mycursor.execute(sql,var)
mydb.commit()
M_code = doc.getElementsByTagName("RiskModel")[1].childNodes[0].getAttribute("_code")
Score= doc.getElementsByTagName("RiskModel")[1].childNodes[1].firstChild.wholeText
ScoreFactorCodeOne = doc.getElementsByTagName("RiskModel")[1].childNodes[2].firstChild.wholeText
ScoreFactorCodeTwo = doc.getElementsByTagName("RiskModel")[1].childNodes[3].firstChild.wholeText
ScoreFactorCodeThree = doc.getElementsByTagName("RiskModel")[1].childNodes[4].firstChild.wholeText
ScoreFactorCodeFour = doc.getElementsByTagName("RiskModel")[1].childNodes[5].firstChild.wholeText
E_code = doc.getElementsByTagName("RiskModel")[1].childNodes[6].getAttribute("_code")
sql = """INSERT INTO RiskModel (TransactionId,M_code,Score,ScoreFactorCodeOne,ScoreFactorCodeTwo,ScoreFactorCodeThree,ScoreFactorCodeFour,E_code) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
var = (TransactionId,M_code,Score,ScoreFactorCodeOne,ScoreFactorCodeTwo,ScoreFactorCodeThree,ScoreFactorCodeFour,E_code)
mycursor.execute(sql,var)
mydb.commit()